# StockTeller Analyzer

This notebook demonstrates stock data analysis and prediction.

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib scikit-learn yfinance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
import yfinance as yf

# Set plot style
plt.style.use('ggplot')

## Define Functions for Stock Analysis

In [ ]:
# Function to load stock data
def load_stock_data(symbol, period='1mo'):
    """
    Load stock data from Yahoo Finance
    """
    try:
        # Try to get data from Yahoo Finance
        data = yf.download(symbol, period=period)
        # Create a simpler dataframe
        stock_data = pd.DataFrame({
            'date': data.index,
            'price': data['Close'],
            'volume': data['Volume'],
        })
        stock_data.reset_index(inplace=True, drop=True)
        
        # Calculate daily change
        stock_data['change'] = stock_data['price'].diff()
        stock_data['changePercent'] = stock_data['price'].pct_change() * 100
        
        # Fill NaN values in first row
        stock_data.loc[0, 'change'] = 0
        stock_data.loc[0, 'changePercent'] = 0
        
        # Add symbol column
        stock_data['symbol'] = symbol
        
        return stock_data
    except Exception as e:
        print(f"Error fetching data from Yahoo Finance: {e}")
        # Generate mock data if there's an error
        return generate_mock_stock_data(symbol)

# Function to generate mock stock data
def generate_mock_stock_data(symbol, days=30):
    """
    Generate mock stock data for demonstration
    """
    today = datetime.now()
    dates = [today - timedelta(days=i) for i in range(days)]
    dates.reverse()  # Oldest first
    
    # Start with a random price
    base_price = np.random.uniform(50, 550)
    prices = [base_price]
    
    # Generate price series with some randomness
    for i in range(1, days):
        change = np.random.normal(0, 0.02) * prices[-1]  # 2% standard deviation
        new_price = max(prices[-1] + change, 1)  # Ensure price doesn't go below 1
        prices.append(new_price)
    
    # Generate volumes
    volumes = np.random.randint(500000, 10000000, size=days)
    
    # Calculate changes
    changes = [0]  # First day has no change
    for i in range(1, days):
        changes.append(prices[i] - prices[i-1])
    
    # Calculate percent changes
    pct_changes = [0]  # First day has no change
    for i in range(1, days):
        pct_changes.append((changes[i] / prices[i-1]) * 100)
    
    # Create DataFrame
    data = pd.DataFrame({
        'date': dates,
        'price': prices,
        'volume': volumes,
        'change': changes,
        'changePercent': pct_changes,
        'symbol': symbol
    })
    
    return data

# Function to predict stock prices
def predict_stock_prices(data, days_to_predict=7):
    """
    Predict future stock prices based on historical data
    """
    # Make a copy of the data to avoid modifying the original
    df = data.copy()
    
    # Get the last date and price from our data
    last_date = df['date'].iloc[-1]  # Already a datetime object
    last_price = df['price'].iloc[-1]
    
    # Prepare data for prediction
    X = np.array(range(len(df))).reshape(-1, 1)
    y = df['price'].values
    
    # Fit linear regression model
    model = LinearRegression()
    model.fit(X, y)
    
    # Prepare prediction dataframe
    predictions = []
    
    # Generate prediction dates (skip weekends)
    for i in range(1, days_to_predict + 1):
        next_date = last_date + timedelta(days=i)
        
        # Skip weekends
        while next_date.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
            next_date = next_date + timedelta(days=1)
        
        # Add some randomness to the prediction
        future_X = np.array([[len(df) + i - 1]])  # Adjust for 0-based indexing
        predicted_price = model.predict(future_X)[0]
        
        # Add some random variation (±2%)
        random_factor = np.random.uniform(0.98, 1.02)
        predicted_price = predicted_price * random_factor
        
        # Calculate change
        if i == 1:
            change = predicted_price - last_price
        else:
            change = predicted_price - predictions[-1]['price']
        
        # Calculate percent change
        if i == 1:
            pct_change = (change / last_price) * 100
        else:
            pct_change = (change / predictions[-1]['price']) * 100
            
        # Add to predictions list
        predictions.append({
            'date': next_date,
            'price': predicted_price,
            'volume': np.random.randint(500000, 10000000),
            'change': change,
            'changePercent': pct_change,
            'symbol': df['symbol'].iloc[0],
            'isPredicted': True
        })
    
    # Convert to DataFrame
    prediction_df = pd.DataFrame(predictions)
    
    return prediction_df

# Function to visualize stock data
def visualize_stock_data(stock_data, predicted_data=None, ticker='AAPL'):
    """
    Create visualization for stock data and predictions
    """
    plt.figure(figsize=(14, 10))
    
    # Plot price
    plt.subplot(2, 1, 1)
    plt.plot(stock_data['date'], stock_data['price'], label='Historical Price', color='blue')
    
    # Plot moving averages
    ma7 = stock_data['price'].rolling(window=7).mean()
    ma30 = stock_data['price'].rolling(window=30).mean() if len(stock_data) >= 30 else None
    
    plt.plot(stock_data['date'], ma7, label='7-Day MA', color='orange')
    if ma30 is not None:
        plt.plot(stock_data['date'], ma30, label='30-Day MA', color='green')
    
    # Plot predictions if available
    if predicted_data is not None and not predicted_data.empty:
        plt.plot(predicted_data['date'], predicted_data['price'], 'r--', label='Predicted Price')
        
        # Shade the prediction area
        plt.axvspan(predicted_data['date'].min(), predicted_data['date'].max(), 
                    alpha=0.1, color='red')
    
    plt.title(f'{ticker} Stock Price Analysis')
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot volume
    plt.subplot(2, 1, 2)
    plt.bar(stock_data['date'], stock_data['volume'], color='lightblue', alpha=0.7)
    plt.title(f'{ticker} Trading Volume')
    plt.xlabel('Date')
    plt.ylabel('Volume')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print prediction summary if available
    if predicted_data is not None and not predicted_data.empty:
        print("\nPredicted prices for the next days:")
        for i, row in predicted_data.iterrows():
            print(f"{row['date'].strftime('%Y-%m-%d')}: ${row['price']:.2f} (Change: {row['change']:.2f}, {row['changePercent']:.2f}%)")

## Load and Analyze Stock Data

In [ ]:
# Set the ticker to analyze
ticker = 'AAPL'

# Load stock data
print(f"Loading data for {ticker}...")
stock_data = load_stock_data(ticker, period='3mo')

# Display the first few rows
stock_data.head()

In [ ]:
# Generate predictions
print("Generating predictions...")
predicted_data = predict_stock_prices(stock_data, days_to_predict=7)

# Display predicted data
predicted_data

In [ ]:
# Visualize the data and predictions
visualize_stock_data(stock_data, predicted_data, ticker)

## Calculate Key Statistics

In [ ]:
# Calculate some key statistics
latest_price = stock_data['price'].iloc[-1]
max_price = stock_data['price'].max()
min_price = stock_data['price'].min()
avg_volume = stock_data['volume'].mean()
price_change = stock_data['price'].iloc[-1] - stock_data['price'].iloc[0]
price_change_pct = (price_change / stock_data['price'].iloc[0]) * 100

print(f"Latest Price: ${latest_price:.2f}")
print(f"High: ${max_price:.2f}")
print(f"Low: ${min_price:.2f}")
print(f"Average Volume: {avg_volume:,.0f}")
print(f"Price Change: ${price_change:.2f} ({price_change_pct:.2f}%)")

## Export Data to CSV

In [ ]:
# Export to CSV
stock_data.to_csv(f"{ticker}_historical.csv", index=False)
predicted_data.to_csv(f"{ticker}_predictions.csv", index=False)
print(f"Data exported to {ticker}_historical.csv and {ticker}_predictions.csv")

## Try Different Stocks

In [ ]:
# Function to analyze a new stock
def analyze_stock(symbol, period='3mo', days_to_predict=7):
    # Load data
    data = load_stock_data(symbol, period=period)
    
    # Generate predictions
    predictions = predict_stock_prices(data, days_to_predict=days_to_predict)
    
    # Visualize
    visualize_stock_data(data, predictions, symbol)
    
    # Calculate key stats
    latest_price = data['price'].iloc[-1]
    max_price = data['price'].max()
    min_price = data['price'].min()
    avg_volume = data['volume'].mean()
    price_change = data['price'].iloc[-1] - data['price'].iloc[0]
    price_change_pct = (price_change / data['price'].iloc[0]) * 100
    
    print(f"Latest Price: ${latest_price:.2f}")
    print(f"High: ${max_price:.2f}")
    print(f"Low: ${min_price:.2f}")
    print(f"Average Volume: {avg_volume:,.0f}")
    print(f"Price Change: ${price_change:.2f} ({price_change_pct:.2f}%)")
    
    return data, predictions

In [ ]:
# Analyze a different stock
# Uncomment and change the symbol to analyze
# analyze_stock('MSFT')